In [1]:
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE
from imblearn.under_sampling import EditedNearestNeighbours, RepeatedEditedNearestNeighbours, AllKNN, NearMiss, NeighbourhoodCleaningRule, OneSidedSelection, TomekLinks
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score 
from imblearn.combine import SMOTETomek, SMOTEENN
from imblearn.pipeline import make_pipeline

import random
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('../datasets/BankChurners_change.csv', encoding='CP949')
data = data.iloc[:,2:23]
data.drop('Customer_Age', axis=1, inplace=True)
np.random.seed(42)
random.seed(42)

In [3]:
object_columns = data.select_dtypes('object').columns

for i in object_columns:

    lb = LabelEncoder()
    lb.fit(data[i])
    data[i] = lb.transform(data[i])
    
    print(f'category : {np.unique(data[i])}\nclasses : {lb.classes_}\n')

input = data.iloc[:,1:]
target = data.iloc[:,0]

category : [0 1]
classes : ['Attrited Customer' 'Existing Customer']

category : [0 1]
classes : ['F' 'M']

category : [0 1 2 3 4 5 6]
classes : ['College' 'Doctorate' 'Graduate' 'High School' 'Post-Graduate'
 'Uneducated' 'Unknown']

category : [0 1 2 3]
classes : ['Divorced' 'Married' 'Single' 'Unknown']

category : [0 1 2 3 4 5]
classes : ['$120K +' '$40K - $60K' '$60K - $80K' '$80K - $120K' 'Less than $40K'
 'Unknown']

category : [0 1 2 3]
classes : ['Blue' 'Gold' 'Platinum' 'Silver']

category : [0 1 2 3 4]
classes : ['20대' '30대' '40대' '50대' '60대 이상']



## Under Sampling

In [4]:
columns = ['f1_score', 'recall', 'precision', 'accuracy']
df = pd.DataFrame(columns=columns)

sampling = [EditedNearestNeighbours, RepeatedEditedNearestNeighbours, AllKNN, NearMiss,NeighbourhoodCleaningRule, OneSidedSelection,TomekLinks]

for i in sampling:
    
    x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

    ss = StandardScaler()
    x_train_ss = ss.fit_transform(x_train)
    x_test_ss = ss.fit_transform(x_test)

    if i == OneSidedSelection:
        sampling = i(random_state=42)
    else:
        sampling = i()

    x_train_ss, y_train = sampling.fit_resample(x_train_ss, y_train)

    svm = SVC(random_state=42)
    svm.fit(x_train_ss, y_train)
    pred = svm.predict(x_test_ss)

    acc = accuracy_score(y_test, pred)
    recall = recall_score(y_test, pred)
    precision = precision_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    
    df.loc[i.__name__] = [f1, recall, precision, acc]

print("SVM Model's score by Under sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False)

SVM Model's score by Under sampling


,f1_score,recall,precision,accuracy
OneSidedSelection,0.958801,0.979400,0.939052,0.929418
TomekLinks,0.958801,0.979400,0.939052,0.929418
NeighbourhoodCleaningRule,0.958358,0.961742,0.954997,0.929911
EditedNearestNeighbours,0.954519,0.951148,0.957913,0.923988
AllKNN,0.953737,0.946439,0.961148,0.923001
RepeatedEditedNearestNeighbours,0.950907,0.940553,0.961492,0.918559
NearMiss,0.796826,0.679812,0.962500,0.709279


## Over Sampling

In [5]:
columns = ['f1_score', 'recall', 'precision', 'accuracy']
df = pd.DataFrame(columns=columns)

sampling = [SMOTE, BorderlineSMOTE, ADASYN]
strategy_ = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

for i in sampling:
    
    for strategy in strategy_:
        x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

        ss = StandardScaler()
        x_train_ss = ss.fit_transform(x_train)
        x_test_ss = ss.fit_transform(x_test)

        sampling = i(sampling_strategy=strategy, random_state=42)
        x_train_ss, y_train = sampling.fit_resample(x_train_ss, y_train)

        svm = SVC(random_state=42)
        svm.fit(x_train_ss, y_train)
        pred = svm.predict(x_test_ss)

        acc = accuracy_score(y_test, pred)
        recall = recall_score(y_test, pred)
        precision = precision_score(y_test, pred)
        f1 = f1_score(y_test, pred)
        
        df.loc[i.__name__+'('+str(strategy)+')'] = [f1, recall, precision, acc]

print("SVM Model's score by Over sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False, inplace=True)
df.head(10)

SVM Model's score by Over sampling


,f1_score,recall,precision,accuracy
ADASYN(0.3),0.960515,0.966451,0.954651,0.933366
BorderlineSMOTE(0.3),0.959859,0.964097,0.955659,0.932379
SMOTE(0.3),0.959767,0.968805,0.950895,0.931885
SMOTE(0.5),0.958985,0.956445,0.961538,0.931392
SMOTE(0.6),0.958506,0.951736,0.965373,0.930898
SMOTE(0.4),0.958358,0.961742,0.954997,0.929911
SMOTE(0.7),0.958123,0.949382,0.967026,0.930405
ADASYN(0.4),0.957422,0.952913,0.961973,0.928924
BorderlineSMOTE(0.4),0.957088,0.951736,0.962500,0.928430
BorderlineSMOTE(0.5),0.956548,0.945851,0.967489,0.927937


## Combine Sampling

In [7]:
columns = ['f1_score', 'recall', 'precision', 'accuracy']
df = pd.DataFrame(columns=columns)

sampling = [SMOTETomek, SMOTEENN]

for i in sampling:

    x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

    ss = StandardScaler()
    x_train_ss = ss.fit_transform(x_train)
    x_test_ss = ss.fit_transform(x_test)

    sampling = i(random_state=42)
    x_train_ss, y_train = sampling.fit_resample(x_train_ss, y_train)

    svm = SVC(random_state=42)
    svm.fit(x_train_ss, y_train)
    pred = svm.predict(x_test_ss)

    acc = accuracy_score(y_test, pred)
    recall = recall_score(y_test, pred)
    precision = precision_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    
    df.loc[i.__name__] = [f1, recall, precision, acc]

print("SVM Model's score by Combine sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False)

SVM Model's score by Combine sampling


,f1_score,recall,precision,accuracy
SMOTETomek,0.953209,0.935256,0.971865,0.923001
SMOTEENN,0.934447,0.889347,0.984365,0.895360


## Pipeline을 통한 Combine Sampling

In [6]:
strategy_ = [0.3, 0.4, 0.5]
columns = ['f1_score', 'recall', 'precision', 'accuracy']

df = pd.DataFrame(columns=columns)

under_sampling = [EditedNearestNeighbours, RepeatedEditedNearestNeighbours, AllKNN, NearMiss,NeighbourhoodCleaningRule, OneSidedSelection,TomekLinks]
over_sampling = [SMOTE, BorderlineSMOTE, ADASYN]

svm = SVC()

for over in over_sampling:

    for under in under_sampling:

        x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

        for strategy in strategy_:
            
            if under == OneSidedSelection:
                under_ = under(random_state=42)
            else:
                under_ = under()

            over_ = over(sampling_strategy=strategy, random_state=42)

            ss = StandardScaler()
            x_train_ss = ss.fit_transform(x_train)
            x_test_ss = ss.fit_transform(x_test)

            model = make_pipeline(over_, under_, svm)   
            model.fit(x_train_ss, y_train)
            pred = model.predict(x_test_ss)

            acc = accuracy_score(y_test, pred)
            recall = recall_score(y_test, pred)
            precision = precision_score(y_test, pred)
            f1 = f1_score(y_test, pred)

            df.loc[over.__name__+under.__name__+'('+str(strategy)+')'] = [f1, recall, precision, acc]

print("SVM Model's score by sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False, inplace=True)
df.head(10)

SVM Model's score by sampling


,f1_score,recall,precision,accuracy
SMOTEOneSidedSelection(0.3),0.960910,0.969394,0.952574,0.933860
SMOTETomekLinks(0.3),0.960910,0.969394,0.952574,0.933860
ADASYNOneSidedSelection(0.3),0.959602,0.964685,0.954572,0.931885
ADASYNTomekLinks(0.3),0.959602,0.964685,0.954572,0.931885
SMOTEOneSidedSelection(0.5),0.958985,0.956445,0.961538,0.931392
BorderlineSMOTEOneSidedSelection(0.3),0.958920,0.961742,0.956115,0.930898
BorderlineSMOTETomekLinks(0.3),0.958920,0.961742,0.956115,0.930898
SMOTETomekLinks(0.5),0.958370,0.955268,0.961493,0.930405
SMOTEOneSidedSelection(0.4),0.957490,0.961154,0.953855,0.928430
SMOTETomekLinks(0.4),0.957490,0.961154,0.953855,0.928430


## Under, Over, Combine Sampling과 Pipeline을 통한 Combine Sampling 전체 결과

In [8]:
sampling = [EditedNearestNeighbours,RepeatedEditedNearestNeighbours, AllKNN, NearMiss,NeighbourhoodCleaningRule, OneSidedSelection,TomekLinks, SMOTE, BorderlineSMOTE, ADASYN, SMOTETomek, SMOTEENN]

for i in sampling:
    x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

    ss = StandardScaler()
    x_train_ss = ss.fit_transform(x_train)
    x_test_ss = ss.fit_transform(x_test)

    if i in [OneSidedSelection, SMOTE, BorderlineSMOTE, ADASYN, SMOTETomek, SMOTEENN]:
        sampling = i(random_state=42)
    else:
        sampling = i()
        
    x_train_ss, y_train = sampling.fit_resample(x_train_ss, y_train)

    svm = SVC(random_state=42)
    svm.fit(x_train_ss, y_train)
    pred = svm.predict(x_test_ss)

    acc = accuracy_score(y_test, pred)
    recall = recall_score(y_test, pred)
    precision = precision_score(y_test, pred)
    f1 = f1_score(y_test, pred)

    df.loc[i.__name__] = [f1, recall, precision, acc]

print("SVM Model's score by sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False, inplace=True)
df.head(10)

SVM Model's score by sampling


,f1_score,recall,precision,accuracy
OneSidedSelection,0.958801,0.979400,0.939052,0.929418
TomekLinks,0.958801,0.979400,0.939052,0.929418
NeighbourhoodCleaningRule,0.958358,0.961742,0.954997,0.929911
EditedNearestNeighbours,0.954519,0.951148,0.957913,0.923988
AllKNN,0.953737,0.946439,0.961148,0.923001
SMOTETomek,0.953209,0.935256,0.971865,0.923001
SMOTE,0.953209,0.935256,0.971865,0.923001
RepeatedEditedNearestNeighbours,0.950907,0.940553,0.961492,0.918559
BorderlineSMOTE,0.947687,0.922307,0.974502,0.914610
ADASYN,0.945897,0.915833,0.978001,0.912142


## No Sampling

In [9]:
columns = ['f1_score', 'recall', 'precision', 'accuracy']
NO_Sampling = pd.DataFrame(columns=columns)

x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

ss = StandardScaler()
x_train_ss = ss.fit_transform(x_train)
x_test_ss = ss.fit_transform(x_test)


svm = SVC(random_state=42)
svm.fit(x_train, y_train)
pred = svm.predict(x_test)

acc = accuracy_score(y_test, pred)
recall = recall_score(y_test, pred)
precision = precision_score(y_test, pred)
f1 = f1_score(y_test, pred)

NO_Sampling.loc['No Sampling'] = [f1, recall, precision, acc]

print("SVM Model's score by sampling")
NO_Sampling.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False, inplace=True)
NO_Sampling.head(10)

SVM Model's score by sampling


,f1_score,recall,precision,accuracy
No Sampling,0.912215,1.0,0.838598,0.838598
